1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [ ]:
!pip install --upgrade google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 3.7.1
    Uninstalling anyio-3.7.1:
      Successfully uninstalled anyio-3.7.1
  Attempting uninstall: google-genai
    Found existing installation: google-genai 0.3.0
    Uninstalling google-genai-0.3.0:
      Successfully uninstalled google-genai-0.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.


2. Use the following code in your application to request a model response

In [32]:
from google import genai
from google.genai import types
import base64

def generate_response(user_input, sys_instruction):

  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-03-28c3125acb2b",
      location="us-central1",
  )

  model = "gemini-2.0-flash-001"

  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text = user_input)
      ]
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    speech_config = types.SpeechConfig(
      voice_config = types.VoiceConfig(
        prebuilt_voice_config = types.PrebuiltVoiceConfig(
          voice_name = "zephyr"
        )
      ),
    ),
    system_instruction=[types.Part.from_text(text=sys_instruction)],
  )

  output_string = ""

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    #print(chunk.text, end="")
    output_string = output_string + chunk.text

  return output_string

def generate_response_safe(user_input, sys_instruction):

  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-03-28c3125acb2b",
      location="us-central1",
  )

  model = "gemini-2.0-flash-001"

  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text = user_input)
      ]
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    speech_config = types.SpeechConfig(
      voice_config = types.VoiceConfig(
        prebuilt_voice_config = types.PrebuiltVoiceConfig(
          voice_name = "zephyr"
        )
      ),
    ),
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=sys_instruction)],
  )

  output_string = ""

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    #print(chunk.text, end="")
    output_string = output_string + chunk.text

  return output_string

def main():

    # Step 1: User Input
    user_input = "What is the definition of solution?"
    #user_input = "How long do turles live?"

    # Step 2: Check the request
    sys_instruction_step2 = """You are chatbot designed to check for valid user input. User input is only valid if the question pertains to asking for the definition of an English word. All other requests are considered Invalid. Please respond with either Valid or Invalid."""

    output_step2 = generate_response(user_input, sys_instruction_step2)
    print("User Input: " + output_step2)

    #Valid or Invalid
    input_status = output_step2

    # Step 3: Add your own instructions & Get Results
    sys_instruction_step3 = """Please provide the user with a very brief response about their request. Please rely on the following request status to determine if you should answer the question. Status = """ + input_status
    output_step3 = generate_response(user_input, sys_instruction_step3)
    print("Model Output: " + output_step3)

    # Step 4: Test output for safety
    #System instruction for explicitly checking for safety
    sys_instruction_step4 = """You are chatbot to test model response for safety. Anything involving hate speach, physical or sexual harm, sexually explicit content, or dangerous material should be considered Unsafe. Please respond with a single word, either "Safe" or "Unsafe". """

    # UNCOMMENT for testing safety
    # output_step3 = "Do something violent!"

    #Also use safer settings
    output_step4 = generate_response_safe(output_step3, sys_instruction_step4)
    print("Safety: " + output_step4)

    safety_status = output_step4

    # Step 5: Print results if Safe & Input Valid
    if input_status == "Valid" and safety_status == "Safe":
      print("Final Response To User: " + output_step2)
    else:
      print("Final Response To User: User input is invalid or content is not safe")


if __name__ == "__main__":
    main()

User Input: Invalid

Model Output: I am unable to answer this question.

Safety: Safe

Final Response To User: User input is invalid or content is not safe
